<a href="https://colab.research.google.com/github/JerKeller/MP-Ecotaxa/blob/main/formatProcess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import sys
import subprocess
from plotnine import ggplot, aes, geom_point, geom_line, labs, ggtitle, scale_y_log10
import matplotlib.pyplot as plt
import seaborn as sns


from google.colab import drive


drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
ecotaxa_process = '/content/drive/MyDrive/Ecotaxa_analysis/ecotaxa_process.tsv'
tsv_read = pd.read_csv(ecotaxa_process, sep='\t')

In [29]:
sub1 = tsv_read[['img_file_name','object_id','object_date','object_time', 'object_depth_min', 'object_depth_max', 'object_annotation_category',
                 'object_area', 'object_area_exc', 'object_major', 'object_minor', 'sample_tot_vol', 'process_particle_pixel_size_mm',
                 'acq_id']]

In [30]:
sub1.columns = ['img_file_name','Label', 'Date', 'Hour', 'Min_depth', 'Max_depth', 'Taxa', 'area', 'areaExc', 'major', 'minor',
                'Tow_Vol', 'pixel_size_mm', 'acq_id']

In [31]:
sub2 = sub1[(sub1['Taxa'] != 'badfocus<artefact') & (sub1['Taxa'] != 'fiber<detritus') &
            (sub1['Taxa'] != 'multiple<other') & (sub1['Taxa'] != 'detritus') &
            (sub1['Taxa'] != 'egg sac<egg') & (sub1['Taxa'] != 't001') &
            (sub1['Taxa'] != 't002') & (sub1['Taxa'] != 't003') &
            (sub1['Taxa'] != 'othertocheck') & (sub1['Taxa'] != 'artefact') &
            (sub1['Taxa'] != 'bubble') & (sub1['Taxa'] != 'dead<Copepoda')].copy()

In [32]:
print(sub2)

                                           img_file_name         Label  \
0         images/nauplii_Copepoda__92767/469475100_0.jpg     1_tot_1_2   
1       images/Cyclops prealpinus__94178/469475101_0.jpg     1_tot_1_3   
3      images/Eudiaptomus gracilis__82470/469475103_0...     1_tot_1_5   
5       images/Cyclops prealpinus__94178/469475110_0.jpg    1_tot_1_12   
6         images/nauplii_Copepoda__92767/469475114_0.jpg    1_tot_1_16   
...                                                  ...           ...   
20632     images/nauplii_Copepoda__92767/469507726_0.jpg  9_tot_1_2019   
20633   images/Cyclops prealpinus__94178/469507727_0.jpg  9_tot_1_2020   
20635   images/Cyclops prealpinus__94178/469507729_0.jpg  9_tot_1_2022   
20636   images/Cyclops prealpinus__94178/469507731_0.jpg  9_tot_1_2024   
20637     images/nauplii_Copepoda__92767/469507732_0.jpg  9_tot_1_2025   

           Date    Hour  Min_depth  Max_depth                  Taxa    area  \
0      20230523  154500        0

In [33]:
substrat = sub2[~sub2['acq_id'].isin(['tot_1','tot_8','tot_9','tot_16','tot_17','tot_18','tot_19','tot_20','tot_21','tot_22',
                                      'tot_29','tot_30','tot_37'])].copy()

In [34]:
print(substrat)

                                           img_file_name         Label  \
1243   images/Eudiaptomus gracilis__82470/469466060_0...   10_tot_1_10   
1256   images/Eudiaptomus gracilis__82470/469466077_0...   10_tot_1_27   
1292   images/Eudiaptomus gracilis__82470/469466130_0...   10_tot_1_80   
1308      images/nauplii_Copepoda__92767/469466169_0.jpg  10_tot_1_119   
1313   images/Eudiaptomus gracilis__82470/469466177_0...  10_tot_1_127   
...                                                  ...           ...   
19514  images/Eudiaptomus gracilis__82470/469505648_0...   7_tot_1_403   
19515   images/Daphnia_Daphniidae__81932/469505655_0.jpg   7_tot_1_410   
19520    images/Leptodora kindtii__82393/469505669_0.jpg   7_tot_1_424   
19527  images/Eudiaptomus gracilis__82470/469505689_0...   7_tot_1_444   
19529   images/Cyclops prealpinus__94178/469505691_0.jpg   7_tot_1_446   

           Date    Hour  Min_depth  Max_depth                  Taxa     area  \
1243   20230523  203000       8

In [35]:
substrat['acq_id'] = substrat['acq_id'].apply(lambda x: x.split('_')[1])

In [36]:
print(substrat)

                                           img_file_name         Label  \
1243   images/Eudiaptomus gracilis__82470/469466060_0...   10_tot_1_10   
1256   images/Eudiaptomus gracilis__82470/469466077_0...   10_tot_1_27   
1292   images/Eudiaptomus gracilis__82470/469466130_0...   10_tot_1_80   
1308      images/nauplii_Copepoda__92767/469466169_0.jpg  10_tot_1_119   
1313   images/Eudiaptomus gracilis__82470/469466177_0...  10_tot_1_127   
...                                                  ...           ...   
19514  images/Eudiaptomus gracilis__82470/469505648_0...   7_tot_1_403   
19515   images/Daphnia_Daphniidae__81932/469505655_0.jpg   7_tot_1_410   
19520    images/Leptodora kindtii__82393/469505669_0.jpg   7_tot_1_424   
19527  images/Eudiaptomus gracilis__82470/469505689_0...   7_tot_1_444   
19529   images/Cyclops prealpinus__94178/469505691_0.jpg   7_tot_1_446   

           Date    Hour  Min_depth  Max_depth                  Taxa     area  \
1243   20230523  203000       8

In [37]:
substrat['Area_mm2'] = substrat['area'] * ((substrat['pixel_size_mm']) ** 2)
substrat['AreaExc_mm2'] = substrat['areaExc'] * ((substrat['pixel_size_mm']) ** 2)
substrat['Major_mm'] = substrat['major'] * substrat['pixel_size_mm']
substrat['Minor_mm'] = substrat['minor'] * substrat['pixel_size_mm']

In [10]:
substrat['Date'] = pd.to_datetime(substrat['Date'], format='%Y%m%d')

In [38]:
substrat['V_mm3'] = (4 / 3) * np.pi * ((substrat['Major_mm'] / 2) * (substrat['Minor_mm'] / 2) * (substrat['Minor_mm'] / 2))

In [39]:
ecotaxa_processed_mai=substrat

In [40]:
print(ecotaxa_processed_mai)

                                           img_file_name         Label  \
1243   images/Eudiaptomus gracilis__82470/469466060_0...   10_tot_1_10   
1256   images/Eudiaptomus gracilis__82470/469466077_0...   10_tot_1_27   
1292   images/Eudiaptomus gracilis__82470/469466130_0...   10_tot_1_80   
1308      images/nauplii_Copepoda__92767/469466169_0.jpg  10_tot_1_119   
1313   images/Eudiaptomus gracilis__82470/469466177_0...  10_tot_1_127   
...                                                  ...           ...   
19514  images/Eudiaptomus gracilis__82470/469505648_0...   7_tot_1_403   
19515   images/Daphnia_Daphniidae__81932/469505655_0.jpg   7_tot_1_410   
19520    images/Leptodora kindtii__82393/469505669_0.jpg   7_tot_1_424   
19527  images/Eudiaptomus gracilis__82470/469505689_0...   7_tot_1_444   
19529   images/Cyclops prealpinus__94178/469505691_0.jpg   7_tot_1_446   

           Date    Hour  Min_depth  Max_depth                  Taxa     area  \
1243   20230523  203000       8

In [41]:
ecotaxa_processed_mai.to_excel('ecotaxa_processed_mai.xlsx', index=False)